# Import Packages

In [1]:
!pip install pandas_datareader

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import pandas_datareader as data
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Reading data from yahoo finance

In [3]:
#S&P500 =sp
sp= data.DataReader("^GSPC",
                       start='2016-1-1', 
                       end='2021-5-25', 
                       data_source='yahoo')
#Stock (Nike)= st
st= data.DataReader("NKE",
                       start='2016-1-1', 
                       end='2021-5-25', 
                       data_source='yahoo')
#Wilshire 5000 index
wls=data.DataReader("^W5000",
                       start='2016-1-1', 
                       end='2021-5-25', 
                       data_source='yahoo')
#Russell 1000 value index
rlv=data.DataReader("^RLV",
                       start='2016-1-1', 
                       end='2021-5-25', 
                       data_source='yahoo')
#Risk-free rate (Rf)
rf=data.DataReader("^IRX",
                       start='2016-1-1', 
                       end='2021-5-25', 
                       data_source='yahoo')

ConnectionError: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Max retries exceeded with url: /quote/%5EGSPC/history?period1=1451620800&period2=1622001599&interval=1d&frequency=1d&filter=history (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fecd6d34040>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

$R = 365*ln (p_t/p_{t-1})$

In [15]:
#Stock returns
R =365*np.log(st['Adj Close']/st['Adj Close'].shift(1)).dropna()
#Market Index returns: S&P500
M =365*np.log(sp['Adj Close']/sp['Adj Close'].shift(1)).dropna()
#Size index: Wilshire 5000 index
S =365*np.log(wls['Adj Close']/wls['Adj Close'].shift(1)).dropna()
#Value index: Russell 1000 value index 
V =365*np.log(rlv['Adj Close']/rlv['Adj Close'].shift(1)).dropna()
#Risk-free rate returns
Rf =(rf['Adj Close']/100).dropna()

In [16]:
#Determining the mean returns of NIKE, S&P500, Wilshire 5000 index, Russell 1000 value index
name= ['r_n','r_m','r_s','r_v','r_f']
mean=[R.mean(),M.mean(), S.mean(),V.mean(),Rf.mean()]
ret= (name,mean)
ret

(['r_n', 'r_m', 'r_s', 'r_v', 'r_f'],
 [0.2213318208464223,
  0.19281434539869813,
  0.19519557381753774,
  0.13302268067813539,
  0.010222813645885903])

In [17]:
# Determining the volatilites of NIKE stock, S&P500 index, Wilshire 5000 index and Russell 1000 value index
name= ['s_n','s_m','s_s','s_v','s_f']
std=[R.var()**0.5,M.var()**0.5, S.var()**0.5,V.var()**0.5,Rf.var()**0.5]
std= (name,std)
std

(['s_n', 's_m', 's_s', 's_v', 's_f'],
 [6.442245133103492,
  4.3773149805433516,
  4.44603301379896,
  4.457652010346641,
  0.008358817599314233])

# Merging the columns into in one worksheet

In [18]:
dt_M =pd.merge(M,Rf, on='Date', how='left').dropna()
dt =pd.merge(dt_M,R, on='Date', how='left').dropna()
dt_1= pd.merge(dt,S, on ='Date', how='left').dropna()
dta= pd.merge(dt_1,V, on='Date', how='left').dropna()

# Renaming the Row Header

In [19]:
dta_cols=['M','Rf','St','S','V']
dta.columns =dta_cols
dta

,M,Rf,St,S,V
Date,,,,,
2016-01-04,-5.629045,0.00155,-5.768505,-5.673750,-4.756967
2016-01-05,0.733725,0.00205,5.067068,0.674867,0.960959
2016-01-06,-4.818789,0.00205,-5.245099,-5.043475,-5.916716
2016-01-07,-8.754823,0.00190,-9.867127,-9.041746,-8.455889
2016-01-08,-3.977601,0.00190,-6.026039,-4.063790,-4.517930
...,...,...,...,...,...
2021-05-19,-1.075929,0.00005,-7.068576,-1.279263,-4.202521
2021-05-20,3.832292,0.00003,0.850007,4.053303,2.184694
2021-05-21,-0.286229,0.00003,-1.674484,-0.180701,3.541917


# OLS Regression to determine beta under APT (3-factor Model)

In [20]:
#Factor Risk Premium
dta['Rp']= dta['M']-dta['Rf']
dta['Rs'] = dta['S']-dta['M']
dta['Rv']= dta['V']-dta['M']
#X & y Variables defined
X = dta [['Rp','Rs','Rv']]
X = sm.add_constant(X)
y= dta.St-dta.Rf
#OLS model
model = sm.OLS(y,X).fit()
predictions =model.predict(X)
Q = model.summary()
print(Q)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.431
Method:                 Least Squares   F-statistic:                     339.5
Date:                Sun, 27 Jun 2021   Prob (F-statistic):          8.08e-164
Time:                        00:11:40   Log-Likelihood:                -4016.8
No. Observations:                1340   AIC:                             8042.
Df Residuals:                    1336   BIC:                             8062.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0715      0.133      0.538      0.5

In [21]:
#Determining the risk-free rate and factor risk premiums of NIKE, S&P500, Wilshire 5000 index and Russell 1000 value index based on average.
f_m = M.mean()-Rf.mean()
f_s = S.mean()-M.mean()
f_v = V.mean()-M.mean()
r_f= Rf.mean()

In [22]:
#Determining Expected Returns from APT given factor risk premiums
ER = r_f + model.params['Rp']*f_m+model.params['Rs']*f_s+model.params['Rv']*f_v
ER

0.1777202609792306

In [23]:
#Determining Alpha (or excess returns)
Alpha = R.mean()-ER
Alpha

0.0436115598671917